In [2]:
import cobra
import multiprocessing
from cobra.io import save_json_model, load_matlab_model, load_json_model
from multiprocessing import Pool
import os
import pandas as pd
import scipy
import docplex
import cplex
from compy import compy
from diet_adaptation import *

In [ ]:
compy(abun_filepath="test_data_input/normCoverageReduced.csv",
     mod_filepath='test_data_input/AGORA103',
     out_filepath="Python_Models",
     diet_filepath='test_data_input/AverageEU_diet_fluxes.txt')

In [ ]:
# Load in Matlab models from MMT

samp1 = cobra.io.load_matlab_model('Matlab_Models/microbiota_model_samp_SRS011061.mat')
# samp2 = cobra.io.load_matlab_model('Matlab_Models/microbiota_model_samp_SRS011134.mat')
# samp3 = cobra.io.load_matlab_model('Matlab_Models/microbiota_model_samp_SRS012902.mat')
# samp4 = cobra.io.load_matlab_model('Matlab_Models/microbiota_model_samp_SRS054956.mat')

In [3]:
# Load in Matlab models from MMT

samp1diet = cobra.io.load_matlab_model('Matlab_Models/microbiota_model_diet_SRS011061.mat')
# samp2diet = cobra.io.load_matlab_model('Matlab_Models/microbiota_model_diet_SRS011134.mat')
# samp3diet = cobra.io.load_matlab_model('Matlab_Models/microbiota_model_diet_SRS012902.mat')
# samp4diet = cobra.io.load_matlab_model('Matlab_Models/microbiota_model_diet_SRS054956.mat')

No defined compartments in model model. Compartments will be deduced heuristically using regular expressions.
Using regular expression found the following compartments:c, d, fe, u


In [ ]:
# Load in the Python models from compy

samp1py = cobra.io.load_json_model('Python_Models/SRS011061_communitymodel_final.json')
# samp2py = cobra.io.load_json_model('Python_Models/SRS011134_communitymodel_final.json')
# samp3py = cobra.io.load_json_model('Python_Models/SRS012902_communitymodel_final.json')
# samp4py = cobra.io.load_json_model('Python_Models/SRS054956_communitymodel_final.json')

In [21]:
samp1pydiet = cobra.io.load_json_model('results/Diet/pymicrobiota_model_diet_SRS011061.json')
# samp2pydiet = cobra.io.load_json_model('results/Diet/pymicrobiota_model_diet_SRS011134.json')
# samp3pydiet = cobra.io.load_json_model('results/Diet/pymicrobiota_model_diet_SRS012902.json')
# samp4pydiet = cobra.io.load_json_model('results/Diet/pymicrobiota_model_diet_SRS054956.json')

KeyboardInterrupt: 

In [ ]:
# Check model information
info_df = pd.DataFrame(columns=['Model', 'Reactions', 'Metabolites', 'Genes', 'Objective'])
models = [(samp1diet, "samp1"), (samp1pydiet, "samp1py"),]

# create a list to store the data for each model
model_data = []

for model in models:
  # add information from model to the list
  model_data.append({
      'Model': model[1],
      'Reactions': len(model[0].reactions),
      'Metabolites': len(model[0].metabolites),
      'Genes': len(model[0].genes),
      'Compartments': model[0].compartments,
      'Objective': model[0].objective.expression if model[0].objective else None
  })

# create the DataFrame from the list of model data
info_df = pd.DataFrame(model_data)
pd.set_option("display.max_colwidth", None)

info_df

In [ ]:
matlab_model = samp1diet
python_model = samp1pydiet

In [ ]:
# Compare Reactions

matlab_rxns = {rxn.id: rxn for rxn in matlab_model.reactions}
python_rxns = {rxn.id: rxn for rxn in python_model.reactions}

# Direct Comparison
common_rxns = set(matlab_rxns.keys()).intersection(python_rxns.keys())
only_in_matlab_rxns = set(matlab_rxns.keys()) - set(python_rxns.keys())
only_in_python_rxns = set(python_rxns.keys()) - set(matlab_rxns.keys())

# Display Number of Above
print(f"Total Num Reactions in MATLAB: {len(matlab_rxns)}")
print(f"Total Num Reactions in Python: {len(python_rxns)}\n")
print(f"Reactions in both MATLAB and Python models: {len(common_rxns)}")
print(f"Reactions only in MATLAB model: {len(only_in_matlab_rxns)}")
print(f"Reactions only in Python model: {len(only_in_python_rxns)}")
print(sorted([rxn for rxn in only_in_matlab_rxns]))
print(sorted([rxn for rxn in only_in_python_rxns]))

In [ ]:
# Compare Mets

matlab_mets = {met.id: met for met in matlab_model.metabolites}
python_mets = {met.id: met for met in python_model.metabolites}

# Direct Comparison
common_mets = set(matlab_mets.keys()).intersection(python_mets.keys())
only_in_matlab_mets = set(matlab_mets.keys()) - set(python_mets.keys())
only_in_python_mets = set(python_mets.keys()) - set(matlab_mets.keys())

# Display Number of Above
print(f"Total Num mets in MATLAB: {len(matlab_mets)}")
print(f"Total Num mets in Python: {len(python_mets)}\n")
print(f"mets in both MATLAB and Python models: {len(common_mets)}")
print(f"mets only in MATLAB model: {len(only_in_matlab_mets)}")
print(f"mets only in Python model: {len(only_in_python_mets)}")

In [ ]:
matlab_models = [samp1diet, samp1diet, samp1diet, samp1diet]
python_models = [samp1pydiet, samp1pydiet, samp1pydiet, samp1pydiet]


# Asserting that reactions in the corr. models have the same reactants, products, coeffs, and reversability
# using a pandas df to display number of inaccuracies
# pd columns = model, num_reactant_diff, num_product_diff, num_coeff_diff, num_reversability_diff

df = pd.DataFrame(columns=['Matlab_Model', 'Python_model', 'Num_Reactant_Diff', 'Num_Product_Diff', 'Num_Coeff_Diff', 'Num_Reversability_Diff'])
num_reactant_diff = [0, 0, 0, 0]
num_product_diff = [0, 0, 0, 0]
num_coeff_diff = [0, 0, 0, 0]
num_reversability_diff = [0, 0, 0, 0]
num_bounds_diff = [0, 0, 0, 0]

for i in range(len(matlab_models)):
  matlab_model = matlab_models[i]
  python_model = python_models[i]

  for rxn in matlab_model.reactions:
      pyrxn = rxn.id.split('Diet_')[1] if 'Diet_' in rxn.id else rxn.id
      matlab_reactants = [met.id for met in rxn.reactants]
      python_reactants = [met.id for met in python_model.reactions.get_by_id(rxn.id).reactants]
      if set(matlab_reactants) != set(python_reactants):
        num_reactant_diff[i] += 1
        # print(rxn.id)
        # print(sorted(matlab_reactants))
        # print(sorted(python_reactants))

      matlab_products = sorted([met.id for met in rxn.products])
      python_products = sorted([met.id for met in python_model.reactions.get_by_id(rxn.id).products])
      if matlab_products != python_products:
        num_product_diff[i] += 1

      matlab_coeffs = [rxn.get_coefficient(met) for met in rxn.reactants + rxn.products]
      python_coeffs = [python_model.reactions.get_by_id(rxn.id).get_coefficient(met) for met in python_model.reactions.get_by_id(rxn.id).reactants + python_model.reactions.get_by_id(rxn.id).products]
      if sorted(matlab_coeffs) != sorted(python_coeffs):
        num_coeff_diff[i] += 1
        matlab_coeffs = [float(val) for val in matlab_coeffs]
        python_coeffs = [float(val) for val in python_coeffs]
        # print(rxn.id)
        # print(sorted(matlab_coeffs))
        # print(sorted(python_coeffs))

      if rxn.reversibility != python_model.reactions.get_by_id(rxn.id).reversibility:
        num_reversability_diff[i] += 1
        # print(rxn.id)
        # print(rxn.reversibility)
        # print(rxn.reversibility)
        
      if f'{rxn.lower_bound:.2f}' != f'{python_model.reactions.get_by_id(rxn.id).lower_bound:.2f}' or f'{rxn.upper_bound:.2f}' != f'{python_model.reactions.get_by_id(rxn.id).upper_bound:.2f}':
        num_bounds_diff[i] += 1
        print(rxn.id, rxn.bounds, python_model.reactions.get_by_id(rxn.id).bounds)

df['Matlab_Model'] = ['samp1diet', 'samp2diet', 'samp3diet', 'samp4diet']
df['Python_model'] = ['samp1pydiet', 'samp2py', 'samp3pydiet', 'samp4py']
df['Num_Rxns'] = [len(model.reactions) for model in matlab_models]
df['Num_Reactant_Diff'] = num_reactant_diff
df['Num_Product_Diff'] = num_product_diff
df['Num_Coeff_Diff'] = num_coeff_diff
df['Num_Reversability_Diff'] = num_reversability_diff
df['Num_Bounds_Diff'] = num_bounds_diff

df

In [ ]:
matlab_models = [samp1diet, samp1diet, samp1diet, samp1diet]
python_models = [samp1pydiet, samp1pydiet, samp1pydiet, samp1pydiet]

# Asserting that reactions in the corr. models have the same reactants, products, coeffs, and reversability
# using a pandas df to display number of inaccuracies
# pd columns = model, num_reactant_diff, num_product_diff, num_coeff_diff, num_reversability_diff


# check that FBA is adhering to the bounds

data = []  # List to store data for DataFrame

for i in range(1):
    matlab_model = matlab_models[i]
    python_model = python_models[i]

    for rxn in matlab_model.reactions:
        if '[d]' in rxn.id and 'EX_' in rxn.id:
            data.append({
                'Matlab_Model': f'samp{i + 1}',  # Dynamically generate model name
                'Python_model': f'samp{i + 1}py',  # Dynamically generate model name
                'Reaction': rxn.id,
                'matlab_rev': rxn.reversibility,
                'python_rev': python_model.reactions.get_by_id(rxn.id).reversibility,
                'matlab_lower_bound': rxn.lower_bound,
                'python_lower_bound': python_model.reactions.get_by_id(rxn.id).lower_bound,
                'matlab_upper_bound': rxn.upper_bound,
                'python_upper_bound': python_model.reactions.get_by_id(rxn.id).upper_bound,
            })

df = pd.DataFrame(data)  # Create DataFrame from collected data
df


In [ ]:
clean_samp_names, organisms, ex_mets = get_individual_size_name(
    abun_file_path='test_data_input/normCoverageReduced.csv',
    mod_path='test_data_input/AGORA103'
)

In [ ]:
clean_samp_names = ['SRS011061', 'SRS012902', 'SRS011134', 'SRS054956']
organisms = ['Bacteroides_cellulosilyticus_DSM_14838',
  'Parabacteroides_merdae_ATCC_43184',
  'Bacteroides_vulgatus_ATCC_8482',
  'Alistipes_putredinis_DSM_17216',
  'Bacteroides_sp_4_3_47FAA',
  'Bacteroides_caccae_ATCC_43185',
  'Burkholderiales_bacterium_1_1_47',
  'Bacteroides_sp_20_3',
  'Bacteroides_ovatus_ATCC_8483',
  'Bacteroides_sp_2_1_33B']
ex_mets = ['12ppd_S[e]',
  '1hibup_S[e]',
  '1hibupglu_S[e]',
  '1hmdgluc[e]',
  '1ohmdz[e]',
  '26dap_M[e]',
  '2dmmq8[e]',
  '2hatvacid[e]',
  '2hatvacidgluc[e]',
  '2hatvlac[e]',
  '2hatvlacgluc[e]',
  '2hibup_S[e]',
  '2hibupglu_S[e]',
  '2hyoxplac[e]',
  '2obut[e]',
  '2oh_cbz[e]',
  '2oh_cbz_glc[e]',
  '2oh_mtz[e]',
  '2oh_mtz_glc[e]',
  '2omfuc[e]',
  '2omxyl[e]',
  '3ddlhept[e]',
  '3hibup_S[e]',
  '3hibupglu_S[e]',
  '3meacmp[e]',
  '3oh_cbz[e]',
  '3oh_cbz_glc[e]',
  '3oh_dlor[e]',
  '3oh_dlor_glc[e]',
  '3oh_mdea[e]',
  '3oh_mdea_glc[e]',
  '3oh_mxn[e]',
  '3oh_mxn_glc[e]',
  '4abut[e]',
  '4dh_tpno[e]',
  '4dh_tpno_1glc[e]',
  '4hbz[e]',
  '4hmdgluc[e]',
  '4hpro_LT[e]',
  '4oh_dcf[e]',
  '4oh_dcf_glc[e]',
  '4oh_kp[e]',
  '4oh_kp_glc[e]',
  '4oh_levole[e]',
  '4oh_levole_glc[e]',
  '4oh_meth[e]',
  '4oh_meth_glc[e]',
  '4oh_propl[e]',
  '4oh_propl_glc[e]',
  '4oh_trz[e]',
  '4oh_trz_glc[e]',
  '4oh_vcz[e]',
  '4oh_vcz_glc[e]',
  '4ohmdz[e]',
  '5asa[e]',
  '5fura[e]',
  '5oh_sulfp[e]',
  '5oh_sulfp_glc[e]',
  '5ohfvs[e]',
  '5ohfvsglu[e]',
  '6bhglz[e]',
  '6bhglzglc[e]',
  '6ohfvs[e]',
  '6ohfvsglu[e]',
  '7a_czp[e]',
  '7bhglz[e]',
  '7bhglzglc[e]',
  '7oh_efv[e]',
  '7oh_efv_glc[e]',
  '814dioh_efv[e]',
  '814dioh_efv_glc[e]',
  '8oh_efv[e]',
  '8oh_efv_glc[e]',
  'C02528[e]',
  'Lcyst[e]',
  'M03134[e]',
  'N_oh_phtn[e]',
  'N_oh_phtn_glc[e]',
  'R_6oh_warf[e]',
  'R_6oh_warf_glc[e]',
  'R_7oh_warf[e]',
  'R_7oh_warf_glc[e]',
  'R_8oh_warf[e]',
  'R_8oh_warf_glc[e]',
  'S_4oh_warf[e]',
  'S_4oh_warf_glc[e]',
  'S_6oh_warf[e]',
  'S_6oh_warf_glc[e]',
  'Ser_Thr[e]',
  'T_antigen[e]',
  'Tn_antigen[e]',
  'Tyr_ggn[e]',
  'abz_ala_b[e]',
  'ac[e]',
  'ac_amn_b_gly[e]',
  'ac_amn_b_gly_glc[e]',
  'acerA[e]',
  'acgal[e]',
  'acgalglcur[e]',
  'acgalidour2s[e]',
  'acgalidour[e]',
  'acgam[e]',
  'acmp[e]',
  'acmp_glc[e]',
  'acmpglu[e]',
  'acnam[e]',
  'adn[e]',
  'adocbl[e]',
  'ala_L[e]',
  'alaasp[e]',
  'alagln[e]',
  'alaglu[e]',
  'alagly[e]',
  'alahis[e]',
  'alaleu[e]',
  'alathr[e]',
  'algin[e]',
  'alpz_4oh[e]',
  'alpz_4oh_glc[e]',
  'alpz_aoh[e]',
  'alpz_aoh_glc[e]',
  'am14[e]',
  'am14_glc[e]',
  'am1ccs[e]',
  'am1cglc[e]',
  'am5[e]',
  'am5_glc[e]',
  'am6[e]',
  'am6_glc[e]',
  'amio[e]',
  'amio_c[e]',
  'amio_c_glc[e]',
  'amio_glc[e]',
  'amn_b_gly[e]',
  'amn_b_gly_glc[e]',
  'amntd_m6[e]',
  'amntd_m6_glc[e]',
  'amylopect900[e]',
  'amylose300[e]',
  'anzp[e]',
  'apio_D[e]',
  'arab_D[e]',
  'arab_L[e]',
  'arabinan101[e]',
  'arabinogal[e]',
  'arabinoxyl[e]',
  'arabttr[e]',
  'arg_L[e]',
  'arsenb[e]',
  'asn_L[e]',
  'asp_L[e]',
  'atvacid[e]',
  'atvacylgluc[e]',
  'atvethgluc[e]',
  'atvlac[e]',
  'atvlacgluc[e]',
  'bglc[e]',
  'bhpm[e]',
  'bhpm_glc[e]',
  'bilr_355[e]',
  'bilr_M10[e]',
  'bilr_M12[e]',
  'bilr_M14[e]',
  'bilr_M15[e]',
  'bilr_M16[e]',
  'bilr_M7[e]',
  'brv[e]',
  'bsn[e]',
  'bsn_glc[e]',
  'btn[e]',
  'but[e]',
  'butam[e]',
  'butso3[e]',
  'bvu[e]',
  'bz[e]',
  'bz_glc[e]',
  'bzam[e]',
  'bzd[e]',
  'ca2[e]',
  'caribup_s[e]',
  'caribupglu_S[e]',
  'cbl1[e]',
  'cbl2[e]',
  'cbz[e]',
  'cbz_glc[e]',
  'cd2[e]',
  'cd6168[e]',
  'cd6168_glc[e]',
  'cellb[e]',
  'cgly[e]',
  'chlphncl[e]',
  'chlphncl_glc[e]',
  'chol[e]',
  'cholate[e]',
  'cit[e]',
  'cl[e]',
  'clcxb[e]',
  'clcxb_c[e]',
  'clcxb_c_glc[e]',
  'clcxb_glc[e]',
  'clobi_c[e]',
  'clobi_glc[e]',
  'co2[e]',
  'cobalt2[e]',
  'core2[e]',
  'core3[e]',
  'core4[e]',
  'core5[e]',
  'core6[e]',
  'core7[e]',
  'core8[e]',
  'cro4[e]',
  'crvsm1[e]',
  'crvsm23[e]',
  'csn[e]',
  'cspg_a[e]',
  'cspg_a_degr[e]',
  'cspg_ab_rest[e]',
  'cspg_b[e]',
  'cspg_b_degr[e]',
  'cspg_c[e]',
  'cspg_c_degr[e]',
  'cspg_c_rest[e]',
  'ctbt[e]',
  'cu2[e]',
  'cvm1gluc[e]',
  'cvm23gluc[e]',
  'cys_L[e]',
  'cytd[e]',
  'czp[e]',
  'daa[e]',
  'daa_glc[e]',
  'dad_2[e]',
  'dcf[e]',
  'dcf_glc[e]',
  'dchac[e]',
  'dcyt[e]',
  'ddea[e]',
  'ddea_glc[e]',
  'des_astzl[e]',
  'des_astzl_glc[e]',
  'dextran40[e]',
  'dextrin[e]',
  'dfdcytd[e]',
  'dfduri[e]',
  'dgchol[e]',
  'dgsn[e]',
  'dh5fura[e]',
  'dhna[e]',
  'digoxin[e]',
  'digoxin_glc[e]',
  'din[e]',
  'dlb[e]',
  'dlb_glc[e]',
  'dma[e]',
  'dms[e]',
  'dmso[e]',
  'dnpz_5des[e]',
  'dnpz_6des[e]',
  'dnpz_m11[e]',
  'dnpz_m12[e]',
  'dnpz_m13[e]',
  'dnpz_m14[e]',
  'dnpz_m9[e]',
  'doh_etr[e]',
  'doh_etr_glc[e]',
  'doh_vcz[e]',
  'doh_vcz_glc[e]',
  'drib[e]',
  'dsT_antigen[e]',
  'duri[e]',
  'dxo[e]',
  'dxo_glc[e]',
  'efv[e]',
  'efv_glc[e]',
  'eltr[e]',
  'eltr_glc[e]',
  'eltr_m3[e]',
  'eltr_m4[e]',
  'estradiol[e]',
  'estradiolglc[e]',
  'estriol[e]',
  'estriolglc[e]',
  'estrone[e]',
  'estroneglc[e]',
  'ethso3[e]',
  'eztmb[e]',
  'eztmb_glc[e]',
  'f1a[e]',
  'fcsn[e]',
  'fe2[e]',
  'fe3[e]',
  'fol[e]',
  'for[e]',
  'fru[e]',
  'fuc_L[e]',
  'fum[e]',
  'fvs[e]',
  'fvsgluc[e]',
  'fvstet[e]',
  'fvstetglu[e]',
  'gal[e]',
  'galmannan[e]',
  'galt[e]',
  'galur[e]',
  'gam26s[e]',
  'gam[e]',
  'gbbtn[e]',
  'gchola[e]',
  'glc3meacp[e]',
  'glc_D[e]',
  'glcmannan[e]',
  'glcn[e]',
  'glcur[e]',
  'gln_L[e]',
  'gltmn[e]',
  'gltmn_glc[e]',
  'glu_L[e]',
  'gly[e]',
  'glyasn[e]',
  'glyasp[e]',
  'glyb[e]',
  'glyc[e]',
  'glycys[e]',
  'glygln[e]',
  'glyglu[e]',
  'glygn2[e]',
  'glygn4[e]',
  'glygn5[e]',
  'glyleu[e]',
  'glymet[e]',
  'glyphe[e]',
  'glypro[e]',
  'glytyr[e]',
  'gmfl[e]',
  'gmfl_glc[e]',
  'gmfl_mIII[e]',
  'gmfl_mIII_glc[e]',
  'gmfl_mII[e]',
  'gmfl_mII_glc[e]',
  'gmfl_mI[e]',
  'gmfl_mI_glc[e]',
  'gncore1[e]',
  'gncore2[e]',
  'gsn[e]',
  'gtacmp[e]',
  'gthox[e]',
  'gua[e]',
  'gulur[e]',
  'h2[e]',
  'h2o[e]',
  'h2s[e]',
  'h[e]',
  'ha[e]',
  'ha_deg1[e]',
  'ha_pre1[e]',
  'hdca[e]',
  'hexs[e]',
  'hg2[e]',
  'his_L[e]',
  'homogal[e]',
  'hspg[e]',
  'hspg_degr_10[e]',
  'hspg_degr_11[e]',
  'hspg_degr_12[e]',
  'hspg_degr_13[e]',
  'hspg_degr_14[e]',
  'hspg_degr_15[e]',
  'hspg_degr_1[e]',
  'hspg_degr_2[e]',
  'hspg_degr_3[e]',
  'hspg_degr_4[e]',
  'hspg_degr_5[e]',
  'hspg_degr_6[e]',
  'hspg_degr_7[e]',
  'hspg_degr_8[e]',
  'hspg_degr_9[e]',
  'hspg_rest[e]',
  'hst[e]',
  'hst_37_diglc[e]',
  'hst_3_glc[e]',
  'hst_3_s[e]',
  'hst_3glc_7s[e]',
  'hst_7_glc[e]',
  'hst_7_s[e]',
  'hst_7glc_3s[e]',
  'hxan[e]',
  'ibup_S[e]',
  'ibupgluc[e]',
  'idour[e]',
  'ile_L[e]',
  'imn[e]',
  'imn_glc[e]',
  'indole[e]',
  'ins[e]',
  'inulin[e]',
  'inv[e]',
  'inv_m1[e]',
  'isetac[e]',
  'isobut[e]',
  'isocapr[e]',
  'isosorbide_5mn[e]',
  'isosorbide_5mn_glc[e]',
  'isoval[e]',
  'k[e]',
  'kdo[e]',
  'kesto[e]',
  'kestopt[e]',
  'kestottr[e]',
  'kprofen[e]',
  'kprofen_glc[e]',
  'lac_D[e]',
  'lac_L[e]',
  'lactl[e]',
  'lcts[e]',
  'leu_L[e]',
  'levan1000[e]',
  'levanb[e]',
  'levanttr[e]',
  'levantttr[e]',
  'lichn[e]',
  'lmn2[e]',
  'lmn30[e]',
  'lst4exp[e]',
  'lstn1gluc[e]',
  'lstn[e]',
  'lstnm4[e]',
  'lstnm7[e]',
  'lys_L[e]',
  'mal_L[e]',
  'malt[e]',
  'malttr[e]',
  'man[e]',
  'mannan[e]',
  'mannur[e]',
  'mantr[e]',
  'mdz[e]',
  'mdz_glc[e]',
  'mdzglc[e]',
  'melib[e]',
  'meoh[e]',
  'met_D[e]',
  'met_L[e]',
  'metala[e]',
  'metsox_R_L[e]',
  'metsox_S_L[e]',
  'mg2[e]',
  'miso[e]',
  'miso_glc[e]',
  'mn2[e]',
  'mops[e]',
  'mqn7[e]',
  'mqn8[e]',
  'mrphn[e]',
  'mrphn_3glc[e]',
  'mrphn_6glc[e]',
  'mso3[e]',
  'mtz[e]',
  'mtz_glc[e]',
  'na1[e]',
  'nac[e]',
  'ncam[e]',
  'nchlphncl[e]',
  'neopront[e]',
  'nh4[e]',
  'nmn[e]',
  'no2[e]',
  'no3[e]',
  'norval_L[e]',
  'nsldp_m5[e]',
  'nsldp_m5_glc[e]',
  'nverp[e]',
  'nverp_glc[e]',
  'nzp[e]',
  'o2[e]',
  'ocdca[e]',
  'ocdcea[e]',
  'odsm_egltmn[e]',
  'odsm_egltmn_glc[e]',
  'odsm_gltmn[e]',
  'odsm_gltmn_glc[e]',
  'oh_etr[e]',
  'oh_etr_glc[e]',
  'oh_pbl[e]',
  'oh_pbl_glc[e]',
  'olsa[e]',
  'orn[e]',
  'pac[e]',
  'pb[e]',
  'pcresol[e]',
  'peamn[e]',
  'pect[e]',
  'pecticgal[e]',
  'phe_L[e]',
  'pheme[e]',
  'phppa[e]',
  'phppa_glc[e]',
  'phtn_glc[e]',
  'pi[e]',
  'pime[e]',
  'pnto_R[e]',
  'ppa[e]',
  'ppi[e]',
  'pro_L[e]',
  'prob[e]',
  'prob_glc[e]',
  'pront[e]',
  'pront_glc[e]',
  'propl[e]',
  'propl_glc[e]',
  'prx_mII[e]',
  'prx_mII_glc[e]',
  'prx_mI[e]',
  'prx_mI_glc[e]',
  'psics_D[e]',
  'ptrc[e]',
  'ptvst[e]',
  'ptvstgluc[e]',
  'pullulan1200[e]',
  'pvs[e]',
  'pvsgluc[e]',
  'pydam[e]',
  'pydx[e]',
  'pydxn[e]',
  'q8[e]',
  'r406[e]',
  'r406_glc[e]',
  'r529[e]',
  'r529_glc[e]',
  'r788[e]',
  'raffin[e]',
  'regfnb[e]',
  'regfnb_glc[e]',
  'rep[e]',
  'rep_glc[e]',
  'rhamnogalurII[e]',
  'rhamnogalurI[e]',
  'rib_D[e]',
  'ribflv[e]',
  'rmn[e]',
  'rpn_104557[e]',
  'rpn_104557_cb_glc[e]',
  'rpn_96990[e]',
  'rpn_96990_glc[e]',
  'rpn_oh[e]',
  'rpn_oh_glc[e]',
  'rsv[e]',
  'rsvgluc[e]',
  'sT_antigen[e]',
  'sTn_antigen[e]',
  'salcn[e]',
  'sanilamide[e]',
  'sb_611855[e]',
  'sb_y[e]',
  'sbzcoa[e]',
  'sch_488128[e]',
  'sch_57871[e]',
  'sch_57871_glc[e]',
  'ser_L[e]',
  'sfnd_1689[e]',
  'sfnd_1689_glc[e]',
  'sftz[e]',
  'sftz_glc[e]',
  'sheme[e]',
  'simvgluc[e]',
  'smap[e]',
  'smap_glc[e]',
  'smvacid[e]',
  'sn38[e]',
  'sn38g[e]',
  'so3[e]',
  'so4[e]',
  'spmd[e]',
  'spz[e]',
  'spz_glc[e]',
  'spz_sfn[e]',
  'spz_sfn_glc[e]',
  'srv[e]',
  'ssz[e]',
  'stg[e]',
  'stg_m3[e]',
  'stg_m4[e]',
  'strch1[e]',
  'strch2[e]',
  'stys[e]',
  'succ[e]',
  'sucr[e]',
  'sulfac[e]',
  'sulfp[e]',
  'tab[e]',
  'tat[e]',
  'taur[e]',
  'tchola[e]',
  'tdchola[e]',
  'tgz[e]',
  'tgz_glc[e]',
  'thm[e]',
  'thsacmp[e]',
  'thymd[e]',
  'tlf_a[e]',
  'tlf_a_1a[e]',
  'tlf_a_1b[e]',
  'tlf_a_1x[e]',
  'tlf_a_2[e]',
  'tlf_a_2a[e]',
  'tlf_a_3[e]',
  'tlf_a_4[e]',
  'tlf_a_m1[e]',
  'tlf_a_m2[e]',
  'tlf_a_m3[e]',
  'tlf_a_m4[e]',
  'tlf_a_m4a[e]',
  'tlf_a_m5[e]',
  'tlf_a_m5a[e]',
  'tlf_a_m5b[e]',
  'tlf_a_m6[e]',
  'tlf_a_m9[e]',
  'tlms[e]',
  'tlms_glc[e]',
  'tma[e]',
  'tmacmp[e]',
  'tmao[e]',
  'tolcp[e]',
  'tolcp_ac[e]',
  'tolcp_ac_glc[e]',
  'tolcp_am[e]',
  'tolcp_am_glc[e]',
  'tolcp_glc[e]',
  'tpno_1glc_4g[e]',
  'tpno_4g[e]',
  'tpno_4glc[e]',
  'tpnoh[e]',
  'tre[e]',
  'trp_L[e]',
  'tsacmgluc[e]',
  'tsul[e]',
  'ttdca[e]',
  'tym[e]',
  'tyr_L[e]',
  'ura[e]',
  'urea[e]',
  'uri[e]',
  'val_L[e]',
  'xan[e]',
  'xyl_D[e]',
  'xylan[e]',
  'xylottr[e]',
  'xyluglc[e]',
  'zn2[e]']

In [ ]:
exchanges, net_production, net_uptake = simulate_microbiota_models(
    samp_names=clean_samp_names[:1],
    ex_mets=ex_mets,
    model_dir='Python_Models',
    diet_file='test_data_input/AverageEU_diet_fluxes.txt',
    res_path='results',
    lower_bm=0.4,
    upper_bm=1.0,
    solver='cplex'
)

In [ ]:
net_secretion_df, net_uptake_df = collect_flux_profiles(
    samp_names=clean_samp_names,
    exchanges=sorted(exchanges),
    net_production=net_production,
    net_uptake=net_uptake
)

In [ ]:
net_secretion_df.to_csv('python_net_secretion_fluxes.csv', index=True)
net_uptake_df.to_csv('python_net_uptake_fluxes.csv', index=True)

In [ ]:
samp1diet.objective = 'EX_26dap_M[fe]'
samp1diet.optimize()
samp1diet.summary()

In [27]:
model = samp1diet

#  First, get the optimal community biomass value
model.objective = 'EX_microbeBiomass[fe]'
biomass_solution = model.optimize()
optimal_biomass = biomass_solution.objective_value
print(f"Optimal community biomass: {optimal_biomass}")

# Add 99.99% optimality constraint (like mgPipe's FVA does)
constraint_value = 0.0 * optimal_biomass
model.reactions.get_by_id('EX_microbeBiomass[fe]').lower_bound = constraint_value
print(f"Applied constraint: EX_microbeBiomass[fe] >= {constraint_value}")

# Now optimize EX_26dap_M[fe] with the biomass constraint
model.objective = 'EX_26dap_M[fe]'
constrained_solution = model.optimize()
print(f"Constrained EX_26dap_M[fe] flux: {constrained_solution.objective_value}")

Optimal community biomass: 1.0
Applied constraint: EX_microbeBiomass[fe] >= 0.0
Constrained EX_26dap_M[fe] flux: 473.0886445730262


In [ ]:
samp1pydiet.objective = 'EX_26dap_M[fe]'
samp1pydiet.optimize()
samp1pydiet.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
Alistipes_putredinis_DSM_17216_dnarep[c],Alistipes_putredinis_DSM_17216_dreplication,0.02634,0,0.00%
Alistipes_putredinis_DSM_17216_proteinsynth[c],Alistipes_putredinis_DSM_17216_pbiosynthesis,0.02634,0,0.00%
Alistipes_putredinis_DSM_17216_rnatrans[c],Alistipes_putredinis_DSM_17216_rtranscription,0.02634,0,0.00%
Bacteroides_cellulosilyticus_DSM_14838_dnarep[c],Bacteroides_cellulosilyticus_DSM_14838_dreplication,0.09935,0,0.00%
Bacteroides_cellulosilyticus_DSM_14838_proteinsynth[c],Bacteroides_cellulosilyticus_DSM_14838_pbiosynthesis,0.09935,0,0.00%
Bacteroides_cellulosilyticus_DSM_14838_rnatrans[c],Bacteroides_cellulosilyticus_DSM_14838_rtranscription,0.09935,0,0.00%
Bacteroides_cellulosilyticus_DSM_14838_s[c],Bacteroides_cellulosilyticus_DSM_14838_sink_s,0.01039,0,0.00%
Bacteroides_ovatus_ATCC_8483_dnarep[c],Bacteroides_ovatus_ATCC_8483_dreplication,0.01244,0,0.00%
Bacteroides_ovatus_ATCC_8483_proteinsynth[c],Bacteroides_ovatus_ATCC_8483_pbiosynthesis,0.01244,0,0.00%
Bacteroides_ovatus_ATCC_8483_rnatrans[c],Bacteroides_ovatus_ATCC_8483_rtranscription,0.01244,0,0.00%
